# Quality testing design
I will test the quality of an embedding both in terms of the structural properties of the nodes and in terms of the development variables.

In [1]:
import os
os.chdir("../..") # Change to parent directory
from struc2vec.src.Helpers import *
os.chdir("structuralNetworkMigration")

In [2]:
import statsmodels.api as sm
import numpy as np
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA, FactorAnalysis
import re


In [3]:
df_emb_4 = pd.read_excel("data/embeddings/all_nw500_1000_wl80_vs75_pca4.xlsx")[["Node","Embedding"]]
df_emb_3 = pd.read_excel("data/embeddings/all_nw500_1000_wl80_vs75_pca3.xlsx")[["Node","Embedding"]]

In [6]:
G = nx.read_gexf("Graphs/0_directed.gexf")

## Predicting structural properties
To test the quality of the classification, I make a linear regression for dummy variables of the classifications as predictions of each structural property. This will give an estimate of the variance within each classification.

In [7]:
df_meta_4, df_agg_4 = get_meta_data(G, df_emb_4["Node"].values, df_emb_4["Embedding"].values)
df_meta_3, df_agg_3 = get_meta_data(G, df_emb_3["Node"].values, df_emb_3["Embedding"].values)

In [8]:
X_4 = pd.get_dummies(df_meta_4["Embedding"], dtype=int).drop(3, axis=1)
X_3 = pd.get_dummies(df_meta_3["Embedding"],  dtype=int).drop(2, axis=1)

In [9]:
struc_reg_4 = {}
for struc_prop in list(df_meta_4.columns[2:]):
    y = df_meta_4[struc_prop]
    X_4 = sm.add_constant(X_4)
    model = sm.OLS(y,X_4)
    results = model.fit()
    struc_reg_4[struc_prop] = results

In [10]:
struc_reg_3 = {}
for struc_prop in list(df_meta_3.columns[2:]):
    y = df_meta_3[struc_prop]
    X_3 = sm.add_constant(X_3)
    model = sm.OLS(y,X_3)
    results = model.fit()
    struc_reg_3[struc_prop] = results

In [17]:
list_of_df_4 = []

for struc_prop, result in struc_reg_4.items():
    params = result.params
    pvalues = result.pvalues
    index_col = pd.MultiIndex.from_tuples([(struc_prop,"params"), (struc_prop,"pvalues")])
    df = pd.DataFrame(columns=index_col)
    df[struc_prop,"params"] = params
    df[struc_prop,"pvalues"] = pvalues

    list_of_df_4.append(df)



In [18]:
list_of_df_3 = []

for struc_prop, result in struc_reg_3.items():
    params = result.params
    pvalues = result.pvalues
    index_col = pd.MultiIndex.from_tuples([(struc_prop,"params"), (struc_prop,"pvalues")])
    df = pd.DataFrame(columns=index_col)
    df[struc_prop,"params"] = params
    df[struc_prop,"pvalues"] = pvalues

    list_of_df_3.append(df)

In [19]:
pd.concat(list_of_df_4, axis=1).apply(lambda x:round(x, 3))

Clustering         Betweenness_centrality         Closeness_centrality  \
          params pvalues                 params pvalues               params   
const      0.609   0.000                  0.002   0.161                0.510   
0          0.080   0.009                 -0.001   0.495               -0.189   
1         -0.025   0.413                 -0.001   0.664               -0.033   
2         -0.056   0.080                  0.010   0.000                0.016   

              InDegree         OutDegree         Average_weight          
      pvalues   params pvalues    params pvalues         params pvalues  
const   0.000   32.953   0.000    45.907   0.000          1.127   0.873  
0       0.000  -17.726   0.036    -9.346   0.023          1.997   0.825  
1       0.419  -10.297   0.222   -24.385   0.000         23.307   0.010  
2       0.723   54.187   0.000     3.453   0.426         19.107   0.048

In [54]:
pd.concat(list_of_df_3, axis=1).apply(lambda x:round(x, 3)).to_excel("testing/structural_indicators.xlsx")

## Predicting development variables

In [21]:
df_hdi = pd.read_excel("metadata_country/metadata_country_3.xlsx")

In [22]:
df_GDP = pd.read_excel("metadata_country/GDP_2015.xlsx")[["Country Code", 2015]]

In [23]:
df_GDP.columns = ["Node", "GDP"]

In [24]:
df_GDP["GDP"] = [float(x) if x != ".." else np.nan for x in df_GDP["GDP"]]

In [25]:
df_GDP.isnull().sum()

Node    0
GDP     8
dtype: int64

In [26]:
df_hdi.isnull().sum()

Node                 0
Embedding_3          0
child_mortality    216
eys                 35
gnipc               35
hdi                 36
le                  33
mmr                 44
mys                 36
Embedding_4          0
dtype: int64

In [27]:
df_hdi = df_hdi.merge(df_GDP, on = "Node")

In [28]:
hdi_mean = df_hdi.groupby("Embedding_3")["hdi"].mean()
index_min = hdi_mean[hdi_mean == min(hdi_mean)].index.values[0]

In [29]:
for column in df_hdi.columns:
    if "Embedding" in column:
        print(column[-1] == "3")

True
False


In [30]:
def getHDITable(df_hdi,  hdi_list =  ['child_mortality', 'eys', 'gnipc', 'hdi', 'le', 'mmr', 'mys','GDP'] ,n_emb="3"):
    for column in df_hdi.columns:
        if "Embedding" in column:
            if column[-1] != str(n_emb):
                df_hdi.drop(column, axis=1)

    hdi_mean = df_hdi.groupby("Embedding_" + n_emb)["hdi"].mean()
    index_min = hdi_mean[hdi_mean == min(hdi_mean)].index.values[0]

    hdi_reg = {}
    for hdi_stat in list(hdi_list):
        df = df_hdi.dropna(subset=hdi_stat)
        y = df[hdi_stat]
        X_hdi = pd.get_dummies(df["Embedding_" + n_emb],dtype=int).drop(index_min, axis=1)
        X_hdi = sm.add_constant(X_hdi)
        model = sm.OLS(y,X_hdi)
        results = model.fit()
        hdi_reg[hdi_stat] = results

    list_of_df = []

    for hdi_stat, result in hdi_reg.items():
        params = result.params
        pvalues = result.pvalues
        index_col = pd.MultiIndex.from_tuples([(hdi_stat,"params"), (hdi_stat,"pvalues")])
        df = pd.DataFrame(columns=index_col)
        df[hdi_stat,"params"] = params
        df[hdi_stat,"pvalues"] = pvalues

        list_of_df.append(df)
    
    return pd.concat(list_of_df, axis=1).apply(lambda x:round(x, 3))

In [35]:
hdi_stat_3 = getHDITable(df_hdi, n_emb="3")

In [36]:
hdi_stat_4 = getHDITable(df_hdi, n_emb="4")

In [46]:
indicators = {
"eys":"Expected Years of Schooling (years)",
"hdi":"Human Development Index (value)",
"le":"Life Expectancy at Birth (years)",
"mys":"Mean Years of Schooling (years)",
"mmr":"Maternal Mortality Ratio (deaths per 100,000 live births)",
"child_mortality":"Child mortality (%)",
"gnipc":"Gross National Income Per Capita (2017 PPP$)",
"GDP":"GDP"
}

In [47]:
index_tubles = []
for layer0, layer1 in hdi_stat_3.columns:
    index_tubles.append((indicators[layer0],layer1))

In [50]:
new_index = pd.MultiIndex.from_tuples(index_tubles)

In [51]:
hdi_stat_3.columns = new_index

In [53]:
hdi_stat_3.to_excel("testing/hdi_indicators.xlsx")

In [ ]:
indicators = {
"eys":"Expected Years of Schooling (years)",
"hdi":"Human Development Index (value)",
"le":"Life Expectancy at Birth (years)",
"mys":"Mean Years of Schooling (years)",
"mmr":"Maternal Mortality Ratio (deaths per 100,000 live births)",
"child_mortality":"Child mortality (%)",
"gnipc":"Gross National Income Per Capita (2017 PPP$)"
}

## Predicting fertility and mortality

In [5]:
df_emb_3.head(2)

,Node,Embedding
0,AFG,0
1,AUS,0


In [6]:
df_fert = pd.read_excel("data/metadata_country/MortalityFertilityData.xlsx")

In [15]:
df_fert_emb = df_emb_3.merge(df_fert, left_on="Node", right_on="Country Code", how="left")

In [16]:
df_fert_emb = df_fert_emb.drop("Country Code", axis=1).set_index("Node", drop=True)

In [19]:
target_list = list(df_fert_emb.columns)[1:]

In [20]:
target_list

['Fertility rate, total (births per woman)',
 'Mortality rate, adult, female (per 1,000 female adults)',
 'Mortality rate, adult, male (per 1,000 male adults)',
 'Mortality rate, infant (per 1,000 live births)']

In [25]:
def getHDITable(df_hdi,  hdi_list =  ['child_mortality', 'eys', 'gnipc', 'hdi', 'le', 'mmr', 'mys','GDP']):
    # for column in df_hdi.columns:
    #     if "Embedding" in column:
    #         if column[-1] != str(n_emb):
    #             df_hdi.drop(column, axis=1)

    # hdi_mean = df_hdi.groupby("Embedding_" + n_emb)["hdi"].mean()
    # index_min = hdi_mean[hdi_mean == min(hdi_mean)].index.values[0]
    hdi_reg = {}
    for hdi_stat in list(hdi_list):
        df = df_hdi.dropna(subset=hdi_stat)
        y = df[hdi_stat]
        X_hdi = pd.get_dummies(df["Embedding"],dtype=int).drop(2, axis=1)
        X_hdi = sm.add_constant(X_hdi)
        model = sm.OLS(y,X_hdi)
        results = model.fit()
        hdi_reg[hdi_stat] = results

    list_of_df = []

    for hdi_stat, result in hdi_reg.items():
        params = result.params
        pvalues = result.pvalues
        index_col = pd.MultiIndex.from_tuples([(hdi_stat,"params"), (hdi_stat,"pvalues")])
        df = pd.DataFrame(columns=index_col)
        df[hdi_stat,"params"] = params
        df[hdi_stat,"pvalues"] = pvalues

        list_of_df.append(df)
    
    return pd.concat(list_of_df, axis=1).apply(lambda x:round(x, 3))

In [26]:
getHDITable(df_fert_emb, hdi_list = target_list)

Fertility rate, total (births per woman)          \
                                        params pvalues   
const                                    3.446     0.0   
0                                       -1.398     0.0   
1                                       -0.895     0.0   

      Mortality rate, adult, female (per 1,000 female adults)          \
                                                       params pvalues   
const                                            166.319        0.000   
0                                                -91.410        0.000   
1                                                -30.475        0.031   

      Mortality rate, adult, male (per 1,000 male adults)          \
                                                   params pvalues   
const                                            238.911    0.000   
0                                                -97.427    0.000   
1                                                -28.315    0.092   

      Mortality rate, infant (per 1,000 live births)          
                                              params pvalues  
const                                         32.399   0.000  
0                                            -22.412   0.000  
1                                             -6.406   0.068

In [28]:
df_fert_emb.groupby("Embedding").mean()

,"Fertility rate, total (births per woman)","Mortality rate, adult, female (per 1,000 female adults)","Mortality rate, adult, male (per 1,000 male adults)","Mortality rate, infant (per 1,000 live births)"
Embedding,,,,
0,2.047293,74.909014,141.484536,9.987143
1,2.550991,135.843769,210.595942,25.992857
2,3.445531,166.318951,238.911321,32.398765


### For reference

In [ ]:
duncan_prestige = sm.datasets.get_rdataset("Duncan", "carData")
Y = duncan_prestige.data['income']
X = duncan_prestige.data['education']
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                     47.51
Date:                Wed, 13 Nov 2024   Prob (F-statistic):           1.84e-08
Time:                        14:33:51   Log-Likelihood:                -190.42
No. Observations:                  45   AIC:                             384.8
Df Residuals:                      43   BIC:                             388.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.6035      5.198      2.040      0.0